In [1]:
import pandas as pd
import numpy as np
import os
import copy
%matplotlib inline    
import matplotlib.pyplot as plt
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
df = pd.DataFrame(columns = ['T','LX','LY','RX','RY','DistanceL','DistanceR','Subject'])
df1 = pd.DataFrame(columns = ['Subject','DistanceR','DistanceL', 'Gender', 'Label'])
labels = [1, 1, 0, 0] #1: Disabled
gender = [1,0,1,0] #1: Male

In [2]:
#Given a dataframe aaa, this routine computes the count and sum of durations for Fixations and Saccades
def computeAggregate(aaa, aColumn, aColumnList):
    aList=copy.copy(aColumnList)
    aListcc=copy.copy(aColumnList)
    
    #Create temp columns changes in the aColumn passed
    aaa['TempSeqChange'] = aaa[aColumn].ne(aaa[aColumn].shift(1)).astype(int)
    bbb = aaa.loc[aaa['TempSeqChange'] != 0]
    bbb['TempDur'] = bbb['T'].shift(-1) - bbb['T']
    
    #Get the sum of durations, grouped by the column passed (Type, HorDur, VerDur)
    cc = bbb.groupby(['Subject', aColumn])['TempDur'].agg('sum')
    cc = cc.reset_index(level=0, drop=True)
    for a in range(len(aColumnList)):
        aList[a] = aColumn + aColumnList[a] + "Sum"
        aListcc[a] = cc[a] 
    df1 = pd.DataFrame([aListcc], columns = aList)

    #Get the mean of durations, grouped by the column passed (Type, HorDur, VerDur)
    cc = bbb.groupby(['Subject', aColumn])['TempDur'].agg('mean')
    cc = cc.reset_index(level=0, drop=True)
    for a in range(len(aColumnList)):
        aList[a] = aColumn + aColumnList[a] + "Mean"
        aListcc[a] = cc[a] 
    df3 = pd.DataFrame([aListcc], columns = aList)

    #Get the count  of durations, grouped by the column passed (Type, HorDur, VerDur)
    cc = bbb.groupby(['Subject', aColumn])[aColumn].agg('count')
    cc = cc.reset_index(level=0, drop=True)
    for a in range(len(aColumnList)):
        aList[a] = aColumn + aColumnList[a] + "Count"
        aListcc[a] = cc[a]     
    df2 = pd.DataFrame([aListcc], columns = aList)
   
    return pd.concat([df1,df2, df3], axis=1)

In [3]:
#fileFolder = "C:\\Python\\Harinee\\dyslexia"
fileFolder = 'C:\\Harinee\\Capstone2\\Data\\Recording Data'
df2 = pd.DataFrame()
for folderName in os.listdir(fileFolder):
    df1 = pd.DataFrame(columns = ['Subject','DistanceR','DistanceL', 'Gender', 'Label'])
    aaa = pd.read_csv(fileFolder + "\\" +  folderName +  "\\A1R.txt", sep="\t", decimal=",")
    #Compute the distance travelled 
    aaa['DistanceL'] = np.sqrt((aaa['LX'].shift(1) - aaa['LX'])**2 + ((aaa['LY'].shift(1) - aaa['LY'])**2))
    aaa['DistanceR'] = np.sqrt((aaa['RX'].shift(1) - aaa['RX'])**2 + ((aaa['RY'].shift(1) - aaa['RY'])**2))
    #1 Saccade 0 Fixation: If the distance travelled is < 1 (review this) then it is Fixation else Saccade
    aaa['LType'] = aaa['DistanceL'].apply(lambda x: 'F' if x < 1 else 'S')
    aaa['RType'] = aaa['DistanceR'].apply(lambda x: 'F' if x < 1 else 'S')
    #-1 left/Down  1 Right/Up 0 No change: Identify the Left and Right horizontal and vertical direction
    aaa['LHorDir'] = (aaa['LX'].shift(1) - aaa['LX']).apply(lambda x: 'N' if abs(x) < 0.01 else ('R' if x <= 0 else 'L'))
    aaa['RHorDir'] = (aaa['RX'].shift(1) - aaa['RX']).apply(lambda x: 'N' if abs(x) < 0.01 else ('R' if x <= 0 else 'L'))
    aaa['LVerDir'] = (aaa['LY'].shift(1) - aaa['LY']).apply(lambda x: 'N' if abs(x) < 0.01 else ('U' if x <= 0 else 'D'))
    aaa['RVerDir'] = (aaa['RY'].shift(1) - aaa['RY']).apply(lambda x: 'N' if abs(x) < 0.01 else ('U' if x <= 0 else 'D'))
    aaa['Subject'] = folderName
    xxx = aaa[['Subject','DistanceR','DistanceL']].groupby('Subject', group_keys=False).sum()
    xxx['Subject'] = folderName
    #get the Label and Gender from the last character of the foldername
    xxx['Label'] = labels[int(folderName[-1])-1]
    xxx['Gender'] = gender[int(folderName[-1])-1]
    df1 = df1.append(xxx).reset_index(level=0, drop=True)
    df1 = pd.concat([df1, 
               computeAggregate(aaa, 'LType', ['F', 'S']),
               computeAggregate(aaa, 'RType', ['F', 'S']),
               computeAggregate(aaa, 'LHorDir', ['L', 'R', 'N']), 
               computeAggregate(aaa, 'RHorDir', ['L', 'R', 'N']), 
               computeAggregate(aaa, 'LVerDir', ['U', 'D', 'N']), 
               computeAggregate(aaa, 'RVerDir', ['U', 'D', 'N'])], 
              axis=1)  
    df2 = df2.append(df1)
#df2 = df2.reset_index(level=0, drop=True)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [4]:
df2 = df2.reset_index(level=0, drop=True)
df2

,DistanceL,DistanceR,Gender,Label,Subject,LTypeFSum,LTypeSSum,LTypeFCount,LTypeSCount,LTypeFMean,...,LVerDirNMean,RVerDirUSum,RVerDirDSum,RVerDirNSum,RVerDirUCount,RVerDirDCount,RVerDirNCount,RVerDirUMean,RVerDirDMean,RVerDirNMean
0,13413.296257,13580.306540,1,0,111GM3,15320.0,14520.0,171,172,89.590643,...,62.027027,11220.0,9900.0,8860.0,262,235,148,42.824427,42.127660,60.272109
1,8788.682167,8509.062191,0,1,111JA2,22100.0,17840.0,288,288,77.003484,...,43.737024,13740.0,13520.0,12680.0,324,317,302,42.407407,42.784810,41.986755
2,9765.357380,10281.893102,1,1,111RP1,24780.0,15180.0,239,240,103.682008,...,47.918367,14680.0,13300.0,11860.0,326,357,260,45.030675,37.254902,45.791506
3,11950.957324,11461.339153,1,0,112JU3,23920.0,15960.0,243,244,98.436214,...,46.666667,10600.0,13300.0,15980.0,270,360,329,39.259259,36.944444,48.719512
4,4959.743932,4913.022136,1,1,112KA1,29920.0,9760.0,249,249,120.645161,...,33.717579,12420.0,15920.0,11580.0,400,457,355,31.050000,34.912281,32.619718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,5375.769629,5092.591991,0,0,826ES4,30500.0,9160.0,208,208,147.342995,...,35.309446,11520.0,17920.0,10520.0,362,439,309,31.823204,40.913242,34.045307
181,9097.917933,9492.893031,1,1,826PP1,23060.0,16480.0,251,251,92.240000,...,43.690037,16380.0,11740.0,11860.0,351,351,268,46.666667,33.542857,44.253731
182,5342.053356,5227.703584,1,1,826SJ1,25000.0,14960.0,266,266,94.339623,...,42.332155,12500.0,15440.0,12040.0,337,389,267,37.091988,39.793814,45.093633
183,7558.301113,7914.926523,0,1,831OK2,25380.0,14580.0,188,189,135.000000,...,47.209302,13400.0,14680.0,11640.0,319,368,260,42.138365,39.891304,44.769231


In [5]:
df.to_csv (r'C:\Harinee\Capstone2\Data\datawithfeatures.csv', index = None, header=True)

In [6]:
df.to_csv (r'C:\Springboard\Springboard01\Capstone\Capstone 2\Data\datawithfeatures.csv', index = None, header=True)